In [ ]:
'''
【项目07】  城市餐饮店铺选址分析

1、从三个维度“口味”、“人均消费”、“性价比”对不同菜系进行比较，并筛选出可开店铺的餐饮类型
要求：
① 计算出三个维度的指标得分
② 评价方法：
   口味 → 得分越高越好
   性价比 → 得分越高越好
   人均消费 → 价格适中即可
③ 制作散点图，x轴为“人均消费”，y轴为“性价比得分”，点的大小为“口味得分”
   绘制柱状图，分别显示“口味得分”、“性价比得分”
   * 建议用bokeh做图
提示：
① 数据清洗，清除空值、为0的数据
② 口味指标计算方法 → 口味评分字段，按照餐饮类别分组算均值，再做标准化处理
③ 人均消费指标计算方法 → 人均消费字段，按照餐饮类别分组算均值，再做标准化处理
④ 性价比指标计算方法 → 性价比 = （口味 + 环境 + 服务）/人均消费，按照餐饮类别分组算均值，再做标准化处理
⑤ 数据计算之前，检查一下数据分布，去除异常值（以外限为标准）
   * 这里排除了高端奢侈餐饮的数据干扰
⑥ 注意，这里先分别计算三个指标，再合并数据（merge）作图，目的是指标之间的噪音数据不相互影响

2、选择一个餐饮类型，在qgis中做将上海划分成格网空间，结合python辅助做空间指标评价，得到餐饮选址位置
* 课程这里以“素菜馆为例”
课程数据
① net_population.shp → 投影坐标系，上海1km²格网内的人口密度数据
② road.shp → 投影坐标西，上海道路数据
要求：
① 通过空间分析，分别计算每个格网内的几个指标：人口密度指标、道路密度指标、餐饮热度指标、同类竞品指标
② 评价方法：
   人口密度指标 → 得分越高越好
   道路密度指标 → 得分越高越好
   餐饮热度指标 → 得分越高越好
   同类竞品指标 → 得分越低越好
   综合指标 = 人口密度指标*0.4 + 餐饮热度指标*0.3 + 道路密度指标*0.2 +同类竞品指标*0.1
③ 最后得到较好选址的网格位置的中心坐标，以及所属区域
   * 可以用bokeh制作散点图
提示：
① 道路密度指标计算方法 → 网格内道路长度
② 餐饮热度指标计算方法 → 网格内餐饮poi计数
③ 同类竞品指标计算方法 → 网格内素菜馆poi计数
④ 餐饮poi数据记得投影
⑤ 可以以“net_population.shp”为网格基础数据，做空间统计
⑥ 在qgis做空间统计之后，网格数据导出点数据，投影成wgs84地理坐标系，导出excel数据，在python做指标标准化等
⑦ 在bokeh中做散点图时，注意添加一个size字段，通过最终评分来赋值
⑧ 在bokeh中做散点图时，可以给TOP10的点用颜色区分

'''

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

import warnings
warnings.filterwarnings('ignore') 
# 不发出警告

from bokeh.io import output_notebook
output_notebook()
# 导入notebook绘图模块

from bokeh.plotting import figure,show
from bokeh.models import ColumnDataSource
# 导入图表绘制、图标展示模块
# 导入ColumnDataSource模块

import os
os.chdir('C:/Users/Administrator/Desktop/x/')

pd.set_option('display.float_format',lambda x : '%.2f' % x)

Loading BokehJS ...

In [2]:
data = pd.read_excel('catering data.xlsx',sheet = 1)

In [6]:
data = data[['类别','口味','环境','服务','人均消费']]
data.dropna(inplace = True)
data = data[(data['口味']>0)&(data['人均消费']>0)]
data['性价比'] = (data['口味'] + data['环境'] + data['服务']) / data['人均消费']
#数据清洗

def f1():
    fig,axes = plt.subplots(1,3,figsize = (10,4))
    data['口味'].plot.box(ax = axes[0])
    data['人均消费'].plot.box(ax = axes[1])
    data['性价比'].plot.box(ax = axes[2])
#箱型图查看异常值
    
def f2(df,col):
    q1 = df[col].quantile(q = 0.25)
    q3 = df[col].quantile(q = 0.75)
    iqr = q3 - q1
    t1 = q1-1.5*iqr
    t2 = q3+1.5*iqr
    result = df[(df[col] > t1)&(df[col] < t2)]
    return result[['类别',col]]

data_kw = f2(data,'口味')
data_rj = f2(data,'人均消费')
data_xjb = f2(data,'性价比')
#清除异常值

def f3(df,col):
    df_gm = df.groupby('类别').mean()
    df_gm[col + '_norm'] = (df_gm[col] - df_gm[col].min()) / (df_gm[col].max() - df_gm[col].min())
    return df_gm.sort_values(col + '_norm',ascending = False)

data_kw_score = f3(data_kw,'口味')
data_rj_score = f3(data_rj,'人均消费')
data_xjb_score = f3(data_xjb,'性价比')
#标准化

data_final_q1 = pd.merge(data_kw_score,data_rj_score,left_index = True,right_index = True)
data_final_q2 = pd.merge(data_final_q1,data_xjb_score,left_index = True,right_index = True)

data_final_q2.sum()

口味           213.89
口味_norm       12.90
人均消费        2000.99
人均消费_norm     10.28
性价比           12.62
性价比_norm       9.86
dtype: float64

In [23]:
#bokeh作图

from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.models import BoxAnnotation

#读取数据
df = data_final_q2

#将行列名改为英文
df.index.name = 'type'
df['size'] = df[df.columns[1]] * 40
df.columns = ['kw','kw_score','rj','rj_score','xjb','xjb_score','size']

source = ColumnDataSource(data = df)

#设置备注
hover = HoverTool(tooltips = [('类别','@type'),
                              ('口味得分','@kw_score'),
                              ('人均付费','@rj'),
                              ('性价比得分','@xjb_score'),
                              ('人均付费得分','@rj_score')])
#散点图
p1 = figure(plot_width = 800,plot_height = 300,
            x_axis_label = '人均付费',y_axis_label = '性价比得分',
            tools = [hover,'pan,crosshair'])
p1.circle(x = 'rj',y = 'xjb_score',source = source,
          size = 'size',line_color = 'black',line_dash = [6,4],fill_alpha = 0.5)


#柱状图
p2 = figure(plot_width = 800,plot_height = 300,
            title = '口味得分',x_range = df.index.tolist(),
            tools = [hover,'pan,crosshair'])
p2.vbar(x = 'type',top = 'kw_score',width = 0.5,source = source,color = 'red')


p3 = figure(plot_width = 800,plot_height = 300,
            title = '人均付费得分',x_range = df.index.tolist(),
            tools = [hover,'pan,crosshair'])
p3.vbar(x = 'type',top = 'rj_score',width = 0.5,source = source,color = 'green')

#合并展示
p = gridplot([[p1],[p2],[p3]])
show(p)